In [ ]:
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        """
        Arguments:
            d_model: モデルの隠れ層の次元数
            dropout: ドロップアウト率
            max_len: 想定される入力シーケンス最大長
        """

        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Positional Encoding行列[max_len, d_model]の初期化
        pe = torch.zeros(max_len, d_model)

        # 位置情報のベクトル
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 10000^(2i/d_model)の計算
        # 対数空間で計算してからexpで戻すことで数値安定性を確保
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 偶数次元にsin、奇数次元にcosを適用
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # バッチ次元を追加してshapeを[1, max_len, d_model]に変形
        pe = pe.unsqueeze(0)

        # モデルのパラメータとして登録（学習されない）
        # state_dictに保存されるが、勾配計算optimizerの対象にはならない
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Enbeddingされた入力テンソル、shapeは[batch_size, seq_len, d_model]
        
        Returns:
            Positional Encodingが加算されたテンソル、shapeは[batch_size, seq_len, d_model]
        """
        # 入力テンソルの長さに合わせてPositional Encodingをスライスして加算
        x = x + self.pe[:, :x.size(1), :]

        # ドロップアウトを適用して出力
        return self.dropout(x)

In [ ]:
def visualize_pe(d_model: int = 128, max_len: int = 100):
    pe_module = PositionalEncoding(d_model=d_model, max_len=max_len, dropout=0.0)

    # ダミーの入力テンソルを作成
    dummy_input = torch.zeros(1, max_len, d_model)
    output = pe_module(dummy_input)

    # 可視化のために1つ目のバッチを取得
    pe_image = output[0].detach().numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(pe_image, aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.xlabel('Embedding Dimension')
    plt.ylabel('Position')
    plt.title('Positional Encoding Visualization')
    plt.show()

visualize_pe(128, 100)